<a href="https://colab.research.google.com/github/parkminhyung/python-code-for-finance/blob/main/portfolio%20optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- load required packages

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
import plotly.io as pio
pio.renderers.default = "colab"

- set parameters
- we create 5000 random portfolios
- the actual risk-free rate in real market is not "0", but we supposed that its value is 0
- init_asset is amount of asset

In [3]:
tickers = ['AAPL','MSFT','TSLA','AMZN','GOOG','2222.SR']
num_port = 5000 #the number of random portfolio
rf = 0 #risk-free rate, the actual risk free rate is not "0",
init_asset = 100000

- create empty data frame and put stock value into the dataframe
- preprocess na value

In [4]:
df = pd.DataFrame()
for ticker in tickers :
  data = yf.download(
    ticker,
    start  = "2020-01-01",
    end = "2023-01-01")
  price = data[["Adj Close"]]
  price = pd.DataFrame(np.log(price["Adj Close"]) - np.log(price["Adj Close"].shift(1)))
  price.columns = [ticker]
  df = pd.concat([df,price],axis=1)

df = (np.round(df,5)).dropna()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


- calculate portfolios' return and covariance

In [5]:
mean_return = df.mean()
cov_mat = df.cov()*252

- Create Random weights Portfolio

In [6]:
all_wts = pd.DataFrame(np.zeros((num_port, df.shape[1])))
all_wts.columns = [ticker + "_weights"  for ticker in tickers]
for i in range(num_port) :
  all_wts.iloc[i] = np.random.uniform(low=0.0, high=1.0, size=df.shape[1])
  all_wts.iloc[i] = all_wts.iloc[i]/np.sum(all_wts.iloc[i])
all_wts

,AAPL_weights,MSFT_weights,TSLA_weights,AMZN_weights,GOOG_weights,2222.SR_weights
0,0.089408,0.243009,0.194918,0.054407,0.319471,0.098788
1,0.387868,0.094136,0.137820,0.147845,0.048951,0.183380
2,0.157495,0.292075,0.036483,0.150163,0.254840,0.108944
3,0.178693,0.075003,0.063629,0.230714,0.117035,0.334926
4,0.125292,0.200093,0.276520,0.144351,0.226979,0.026765
...,...,...,...,...,...,...
4995,0.119071,0.151994,0.085057,0.216075,0.206299,0.221504
4996,0.041769,0.045045,0.308453,0.082175,0.377889,0.144669
4997,0.223179,0.013702,0.244588,0.212640,0.097199,0.208692
4998,0.266640,0.281036,0.088777,0.059283,0.291090,0.013174


- Create return, risk, sharpe-ratio columns and calculate those values

In [7]:
all_wts["return"], all_wts["risk"], all_wts["sharpe"] = (np.zeros((num_port, 1)) for _ in range(3))
for i in range(num_port) :
  all_wts["return"][i] = pow((1+np.array(all_wts.iloc[i,:len(tickers)]).dot(mean_return)),252)-1
  all_wts["risk"][i] = np.sqrt(np.array(all_wts.iloc[i,:len(tickers)]).dot(cov_mat.dot(np.array(all_wts.iloc[i,:len(tickers)]))))
all_wts["sharpe"] = (all_wts["return"]-rf)/all_wts["risk"]
all_wts

,AAPL_weights,MSFT_weights,TSLA_weights,AMZN_weights,GOOG_weights,2222.SR_weights,return,risk,sharpe
0,0.089408,0.243009,0.194918,0.054407,0.319471,0.098788,0.383348,0.341641,1.122081
1,0.387868,0.094136,0.137820,0.147845,0.048951,0.183380,0.388202,0.309955,1.252447
2,0.157495,0.292075,0.036483,0.150163,0.254840,0.108944,0.252363,0.301241,0.837745
3,0.178693,0.075003,0.063629,0.230714,0.117035,0.334926,0.290292,0.247467,1.173052
4,0.125292,0.200093,0.276520,0.144351,0.226979,0.026765,0.473190,0.388664,1.217478
...,...,...,...,...,...,...,...,...,...
4995,0.119071,0.151994,0.085057,0.216075,0.206299,0.221504,0.295354,0.278579,1.060216
4996,0.041769,0.045045,0.308453,0.082175,0.377889,0.144669,0.490509,0.369487,1.327542
4997,0.223179,0.013702,0.244588,0.212640,0.097199,0.208692,0.468149,0.334098,1.401234
4998,0.266640,0.281036,0.088777,0.059283,0.291090,0.013174,0.310640,0.339537,0.914894


- "tang" is tangency portfolio that has the largest sharpe ratio value
- "risk-min" is minimized risk portfolio that has the smallest risk value

In [8]:
tang = all_wts.iloc[all_wts["sharpe"].idxmax()]
risk_min = all_wts.iloc[all_wts["risk"].idxmin()]


- show weights and allocation of tangency portfolio

In [9]:
print( "[ tangency portfolio % ]" ,"\n" ,tang.iloc[:len(tickers)],"\n","\n",
"[ asset allocation $]","\n",">> Initial Asset : ",init_asset,"\n",tang.iloc[:len(tickers)] * init_asset)


[ tangency portfolio % ] 
 AAPL_weights       0.062583
MSFT_weights       0.012724
TSLA_weights       0.772018
AMZN_weights       0.072975
GOOG_weights       0.017463
2222.SR_weights    0.062236
Name: 1509, dtype: float64 
 
 [ asset allocation $] 
 >> Initial Asset :  100000 
 AAPL_weights        6258.345898
MSFT_weights        1272.423220
TSLA_weights       77201.841296
AMZN_weights        7297.488303
GOOG_weights        1746.309402
2222.SR_weights     6223.591880
Name: 1509, dtype: float64


- show weights and allocation of risk-min portfolio

In [10]:
#global minimize portfolio
print( "[ global minimize portfolio %]" ,"\n",risk_min.iloc[:len(tickers)],"\n","\n",
"[ asset allocation $]","\n",">> Initial Asset : ",init_asset,"\n",risk_min.iloc[:len(tickers)] * init_asset)



[ global minimize portfolio %] 
 AAPL_weights       0.013402
MSFT_weights       0.048892
TSLA_weights       0.039972
AMZN_weights       0.088433
GOOG_weights       0.205544
2222.SR_weights    0.603756
Name: 1239, dtype: float64 
 
 [ asset allocation $] 
 >> Initial Asset :  100000 
 AAPL_weights        1340.215242
MSFT_weights        4889.246092
TSLA_weights        3997.213056
AMZN_weights        8843.328618
GOOG_weights       20554.425647
2222.SR_weights    60375.571345
Name: 1239, dtype: float64


- Visualize random portfolio and create annotations (global minimized portfolio and tangency portfolio)

In [11]:
ann_min = go.layout.Annotation(
    x=risk_min['risk'],
    y=risk_min['return'],
    text="Minimum Variance Portfolio",
    xref="x",
    yref="y",
    showarrow=True,
    arrowhead=0
)

ann_tang = go.layout.Annotation(
    x=tang['risk'],
    y=tang['return'],
    text="Tangency Portfolio",
    xref="x",
    yref="y",
    showarrow=True,
    arrowhead=0
)

# Define layout for titles
title1 = {
    'text': "<b> Portfolio Optimization and Efficient Frontier </b>",
    'xref': "paper",
    'yref': "paper",
    'xanchor': "center",
    'yanchor': "bottom",
    'align': "center",
    'x': 0.5,
    'y': 1,
    'showarrow': False
}

title2 = {
    'text': "<b> Portfolio Weights </b>",
    'xref': "paper",
    'yref': "paper",
    'xanchor': "center",
    'yanchor': "bottom",
    'align': "center",
    'x': 0.5,
    'y': 1,
    'showarrow': False
}

# Create a subplot with two figures
fig1 = go.Figure()
fig1.add_trace(go.Scatter(
    x=all_wts['risk'],
    y=all_wts['return'],
    mode='markers',
    marker=dict(
        color=all_wts['sharpe'],
        colorscale='Viridis',
        showscale=True
    ),
    showlegend=False
))

fig1.add_trace(go.Scatter(
    x=[risk_min['risk']],
    y=[risk_min['return']],
    mode='markers',
    marker=dict(color="#A52929"),
))

fig1.add_trace(go.Scatter(
    x=[tang['risk']],
    y=[tang['return']],
    mode='markers',
    marker=dict(color="#A52929"),
))

fig1.update_xaxes(title="σ", zeroline=False)
fig1.update_yaxes(title="µ", zeroline=False)
fig1.update_layout(annotations=[title1, ann_min, ann_tang], coloraxis_colorbar=dict(title="Sharpe Ratio"))
fig1.show()

- build weights plots from portfolios

In [12]:
fig2 = go.Figure()
fig2.add_trace(go.Bar(
    x= risk_min.iloc[:len(tickers)].index,
    y= risk_min.iloc[:len(tickers)],
    name="Minimum Weights",
    marker=dict(color='#3246AB')
))

fig2.add_trace(go.Bar(
    x=tang.iloc[:len(tickers)].index,
    y=tang.iloc[:len(tickers)],
    name="Tangency Weights",
    marker=dict(color='#12B7C3')
))

fig2.update_layout(annotations=[title2], yaxis=dict(title="Weights"), barmode="group")
fig2.show()

- merge two plots by making subplots

In [13]:
# Create subplots
subplots = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("<b> Portfolio Optimization and Efficient Frontier </b>", "<b> Portfolio Weights </b>"))
for trace in fig1.data : subplots.add_trace(trace, row=1,col=1)
for trace in fig2.data : subplots.add_trace(trace, row=2,col=1)
subplots.update_layout(title_x=0.5, title_y=1, margin=dict(b=0.1))
subplots.show()